# Necessary import

In [169]:
import pandas as pd
import altair as alt

# Load data

In [170]:
# Load the DataFrame
df = pd.read_csv('dpt2020.csv', delimiter=';')

# Remove rows with missing or invalid values
df = df.dropna(subset=['annais', 'sexe'])

# Drop rows with 'XXXX' in the 'annais' column
df = df[df['annais'] != 'XXXX']

In [171]:
df

,sexe,preusuel,annais,dpt,nombre
0,1,_PRENOMS_RARES,1900,02,7
1,1,_PRENOMS_RARES,1900,04,9
2,1,_PRENOMS_RARES,1900,05,8
3,1,_PRENOMS_RARES,1900,06,23
4,1,_PRENOMS_RARES,1900,07,9
...,...,...,...,...,...
3727545,2,ZYA,2013,44,4
3727546,2,ZYA,2013,59,3
3727547,2,ZYA,2017,974,3
3727548,2,ZYA,2018,59,3


# group data by sum

In [172]:
# Group the data by 'preusuel', 'annais', and 'sexe' and calculate the total count for each combination
grouped_data = df.groupby(['preusuel', 'annais', 'sexe'], as_index=False)['nombre'].sum()

# Calculate the total count for each 'preusuel' and 'annais' combination
grouped_data['total_count'] = grouped_data.groupby(['preusuel', 'annais'])['nombre'].transform('sum')

In [173]:
grouped_data

,preusuel,annais,sexe,nombre,total_count
0,AADIL,1983,1,3,3
1,AADIL,1992,1,3,3
2,AAHIL,2016,1,3,3
3,AALIYA,2017,2,3,3
4,AALIYAH,2001,2,9,9
...,...,...,...,...,...
257583,ÖMER,2016,1,18,18
257584,ÖMER,2017,1,30,30
257585,ÖMER,2018,1,31,31
257586,ÖMER,2019,1,37,37


# generate male and female percentage by names

In [174]:
# Calculate the male and female percentages for each 'preusuel' and 'annais' combination
grouped_data['male_percentage'] = grouped_data.apply(lambda row: row['nombre'] / row['total_count'] if row['sexe'] == 1 else 0, axis=1)
grouped_data['female_percentage'] = grouped_data.apply(lambda row: row['nombre'] / row['total_count'] if row['sexe'] == 2 else 0, axis=1)

# Sort the DataFrame by 'annais'
grouped_data = grouped_data.sort_values('annais')

In [175]:
grouped_data

,preusuel,annais,sexe,nombre,total_count,male_percentage,female_percentage
246961,WALTHER,1900,1,8,8,1.0,0.0
114281,JEAN-BAPTISTE,1900,1,241,241,1.0,0.0
12724,AMALIE,1900,2,11,11,0.0,1.0
101715,HERBERT,1900,1,4,4,1.0,0.0
247937,WILHELMINE,1900,2,10,10,0.0,1.0
...,...,...,...,...,...,...,...
43020,CHAMS,2020,1,3,3,1.0,0.0
43075,CHANEL,2020,2,6,6,0.0,1.0
211216,ROKYA,2020,2,3,3,0.0,1.0
212707,ROSA,2020,2,63,63,0.0,1.0


# Generate percentages to show the genra of names

In [176]:
# Define the desired bin ranges for male and female percentages
bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
labels = ['Absolutly female (<90%)', 
          'Mostly female (90%-80%)',
          'Significantly female (80%-70%)',
          'Somewhat female (60%-50%)',
          'Slightly female (60%-50%)', 
          'Slightly male (60%-50%)', 
          'Somewhat male (70%-60%)', 
          'Significantly male (80%-70%)',
          'Mostly male (90%-80%)', 
          'Absolutly male (<90%)']

# Create a new column 'percentage_category' based on male_percentage values
grouped_data['percentage_category'] = pd.cut(grouped_data['male_percentage'], bins=bins, labels=labels)

# Group the data by 'annais' and 'percentage_category' and calculate the sum of 'nombre'
grouped_data = grouped_data.groupby(['annais', 'percentage_category'])['nombre'].sum().reset_index()

# Pivot the data to have years as rows and percentage categories as columns
grouped_data = grouped_data.pivot(index='annais', columns='percentage_category', values='nombre')

# Fill any missing values with 0
grouped_data = pivot_table.fillna(0)

In [177]:
grouped_data

percentage_category,Absolutly female (<90%),Mostly female (90%-80%),Significantly female (80%-70%),Somewhat female (60%-50%),Slightly female (60%-50%),Slightly male (60%-50%),Somewhat male (70%-60%),Significantly male (80%-70%),Mostly male (90%-80%),Absolutly male (<90%)
annais,,,,,,,,,,
1900,1060,6,6,5,1232,0,1200,28,309,33730
1901,1036,17,0,8,1326,0,1296,22,353,40438
1902,1047,18,6,15,1315,0,1298,57,295,59337
1903,1040,15,3,13,1281,0,1305,54,197,24068
1904,980,4,3,8,1428,9,1422,23,315,39224
...,...,...,...,...,...,...,...,...,...,...
2016,128,185,1380,161,24090,31,834,2738,927,12992
2017,147,255,0,1621,24528,26,624,2971,883,11800
2018,191,267,12,1857,24827,201,685,350,3513,14397


# Plot the result in Altair

In [180]:
# Convert pivot_table to long format
grouped_data_long = grouped_data.reset_index().melt(id_vars='annais', var_name='percentage_category', value_name='quantity')

# Create the bar chart
chart = alt.Chart(grouped_data_long).mark_bar().encode(
    x='annais:O',
    y='quantity:Q',
    color='percentage_category:N',
    tooltip=['annais', 'quantity']
).properties(
    width=1500,
    height=400,
    title='Evolution of genre in Unisex names'
)



In [181]:
# Show the chart
chart

alt.Chart(...)